In [1]:
import sys
sys.path.append("../code/")
import numpy as np
import pandas as pd
from dataloader import APPLIANCE_ORDER, get_train_test
from tensor_custom_core import stf_4dim, stf_4dim_time
import torch
import torch.nn as nn
from torch.autograd import Variable
torch.manual_seed(0)
np.random.seed(0)

In [2]:
class CustomRNN(nn.Module):
    def __init__(self, cell_type, hidden_size, num_layers, bidirectional):
        super(CustomRNN, self).__init__()
        torch.manual_seed(0)

        if bidirectional:
            self.num_directions = 2
        else:
            self.num_directions = 1
        if cell_type=="RNN":
            self.rnn = nn.RNN(input_size=1, hidden_size=hidden_size,
                   num_layers=num_layers, batch_first=True,
                   bidirectional=bidirectional)
        elif cell_type=="GRU":
            self.rnn = nn.GRU(input_size=1, hidden_size=hidden_size,
                              num_layers=num_layers, batch_first=True,
                              bidirectional=bidirectional)
        else:
            self.rnn = nn.LSTM(input_size=1, hidden_size=hidden_size,
                              num_layers=num_layers, batch_first=True,
                              bidirectional=bidirectional)

        self.linear = nn.Linear(hidden_size*self.num_directions, 1 )
        self.act = nn.ReLU()

    def forward(self, x):
        pred, hidden = self.rnn(x, None)
        pred = self.linear(pred)
        
        #pred = pred[:, :, 23:24]
        #pred = self.act(pred)
        #pred = torch.clamp(pred, min=0.)
        #pred = self.act(pred)
        #pred = torch.min(pred, x)
        return pred


num_folds = 5

if torch.cuda.is_available():
    cuda_av = True
else:
    cuda_av=False

In [3]:
fold_num = 0
num_folds = 5
cell_type="GRU"
hidden_size = 10
lr = 1
bidirectional = True
appliance = "hvac"

In [5]:
torch.manual_seed(0)

appliance_num = APPLIANCE_ORDER.index(appliance)
train, test = get_train_test(2, num_folds=num_folds, fold_num=fold_num)

In [6]:


train_aggregate = train[:, 0, :, :].reshape(train.shape[0], -1, 1)



test_aggregate = test[:, 0, :, :].reshape(test.shape[0], -1, 1)

train_appliance = train[:, appliance_num, :, :].reshape(train.shape[0], -1, 1)
test_appliance = test[:, appliance_num, :, :].reshape(test.shape[0], -1, 1)
   

In [7]:
train_aggregate.shape, train_appliance.shape

((54, 2688, 1), (54, 2688, 1))

In [8]:
loss_func = nn.L1Loss()
r = CustomRNN(cell_type, hidden_size, 1, bidirectional)

if cuda_av:
    r = r.cuda()
    loss_func = loss_func.cuda()

optimizer = torch.optim.Adam(r.parameters(), lr=lr)

In [9]:
num_iterations=100
for t in range(num_iterations):

    inp = Variable(torch.Tensor(train_aggregate), requires_grad=True)
    train_y = Variable(torch.Tensor(train_appliance))
    if cuda_av:
        inp = inp.cuda()
        train_y = train_y.cuda()
    pred = r(inp)
    print(pred.std().data[0], pred.mean().data[0])
    optimizer.zero_grad()
    loss = loss_func(pred, train_y)
    if t % 1 == 0:
        print(t, loss.data[0])
    loss.backward()
    optimizer.step()
    
    

0.0037488178350031376 -0.05482201278209686
0 892.5844116210938
0.21765337884426117 1.7938494682312012
1 891.1170654296875
0.22589603066444397 7.084446430206299
2 888.0406494140625
0.04937722533941269 16.551998138427734
3 881.9739990234375
0.08351068198680878 25.708831787109375
4 876.16845703125
0.15687118470668793 35.672279357910156
5 869.8837280273438
0.2766116261482239 46.190555572509766
6 864.0235595703125
0.37241050601005554 57.09162902832031
7 857.1917724609375
0.4735007584095001 68.2833023071289
8 850.9346923828125
0.601460337638855 79.70441436767578
9 843.9619140625
0.8192564249038696 91.31637573242188
10 836.8756103515625
1.1024949550628662 103.07072448730469
11 830.4198608398438
1.29531729221344 114.91517639160156
12 823.2904052734375
1.4135112762451172 126.83101654052734
13 816.8997802734375
1.5126649141311646 138.8036651611328
14 809.8945922851562
1.608328938484192 150.81256103515625
15 803.67724609375
1.703594446182251 162.84075927734375
16 796.9068603515625
1.7991117238998

In [ ]:
test_inp = Variable(torch.Tensor(test_aggregate), requires_grad=False)
test_y = Variable(torch.Tensor(test_appliance), requires_grad=False)
if cuda_av:
    test_inp = test_inp.cuda()
pred = r(test_inp)
#pred[pred<0.] = 0.
pred = torch.clamp(pred, min=0.)
if cuda_av:
    prediction_fold = pred.cpu().data.numpy()
else:
    prediction_fold = pred.data.numpy()

In [ ]:
gts = []
preds = []

def disagg_fold(fold_num, appliance, cell_type, hidden_size,
                num_layers, bidirectional, lr,
                num_iterations):
    torch.manual_seed(0)

    appliance_num = APPLIANCE_ORDER.index(appliance)
    train, test = get_train_test(num_folds=num_folds, fold_num=fold_num)
    train_aggregate = train[:, 0, :, :].reshape(-1, 24, 1)
    test_aggregate = test[:, 0, :, :].reshape(-1, 24, 1)

    train_appliance = train[:, appliance_num, :, :].reshape(-1, 24, 1)
    test_appliance = test[:, appliance_num, :, :].reshape(-1, 24, 1)
    gts.append(test_appliance.reshape(-1, 24))
    loss_func = nn.L1Loss()
    r = CustomRNN(cell_type, hidden_size, num_layers, bidirectional)

    if cuda_av:
        r = r.cuda()
        loss_func = loss_func.cuda()

    # Setting the params all to be non-negative
    #for param in r.parameters():
    #    param.data = param.data.abs()

    optimizer = torch.optim.Adam(r.parameters(), lr=lr)

    for t in range(num_iterations):

        inp = Variable(torch.Tensor(train_aggregate), requires_grad=True)
        train_y = Variable(torch.Tensor(train_appliance))
        if cuda_av:
            inp = inp.cuda()
            train_y = train_y.cuda()
        pred = r(inp)

        optimizer.zero_grad()
        loss = loss_func(pred, train_y)
        if t % 5 == 0:
            print(t, loss.data[0])
        loss.backward()
        optimizer.step()

    test_inp = Variable(torch.Tensor(test_aggregate), requires_grad=False)
    test_y = Variable(torch.Tensor(test_appliance), requires_grad=False)
    if cuda_av:
        test_inp = test_inp.cuda()
    pred = r(test_inp)
    #pred[pred<0.] = 0.
    pred = torch.clamp(pred, min=0.)
    if cuda_av:
        prediction_fold = pred.cpu().data.numpy()
    else:
        prediction_fold = pred.data.numpy()
    return prediction_fold, test_appliance

def disagg(appliance, cell_type, hidden_size, num_layers, bidirectional, lr, num_iterations):
    from sklearn.metrics import mean_absolute_error
    preds = []
    gts = []
    for cur_fold in range(num_folds):
        pred, gt = disagg_fold(cur_fold, appliance, cell_type, hidden_size, num_layers
                               ,bidirectional, lr, num_iterations)

        preds.append(pred)
        gts.append(gt)
    return mean_absolute_error(np.concatenate(gts).flatten(), np.concatenate(preds).flatten())

appliance = "hvac"
cell_type="GRU" # One of GRU, LSTM, RNN
hidden_size=100 # [20, 50, 100, 150]
num_layers=1  # [1, 2, 3, 4]
bidirectional=False # True or False
lr =1 # 1e-3, 1e-2, 1e-1, 1, 2
num_iterations = 20 #200, 400, 600, 800

appliance, cell_type, hidden_size, num_layers, bidirectional, lr, num_iterations = sys.argv[1:]
hidden_size = int(hidden_size)
num_layers = int(num_layers)
lr = float(lr)
num_iterations = int(num_iterations)

p = disagg(appliance, cell_type, hidden_size, num_layers,
                bidirectional, lr, num_iterations)

import pickle
pickle.dump(p, open("./baseline/rnn-individual-baseline-result/rnn-individual-{}-{}-{}-{}-{}-{}-{}.pkl".format(appliance,
						cell_type, hidden_size, num_layers, bidirectional, lr, num_iterations), "wb"))